In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_augmentation import random_transform


from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, MaxPooling2D, Reshape, Flatten, Input, merge, subtract
from keras import backend as K

Using TensorFlow backend.


In [2]:
resize_shape = (128, 128, 3)

In [54]:
data = pd.read_csv("data/train.csv")

In [55]:
#For now, we remove new_whale
# data = data[data['Id'] != 'new_whale'].reset_index(drop=True)

In [56]:
len(data)

9850

# Look at example 

In [57]:
data.head(10)

,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3
5,0025e8c2.jpg,w_8b1ca89
6,0026a8ab.jpg,w_eaad6a8
7,0031c258.jpg,new_whale
8,0035632e.jpg,w_3d0bc7a
9,0037e7d3.jpg,w_50db782


In [7]:
from PIL import Image
image = Image.open('data/train/00022e1a.jpg')

In [8]:
np.array(image).shape

(500, 699)

In [9]:
# plt.imshow(image)
# plt.show()

In [10]:
np.array(image).shape

(500, 699)

In [11]:
new_image = np.stack([image]*3,axis=2)

In [12]:
np.array(new_image).shape

(500, 699, 3)

In [13]:
# gray = np.mean(image, -1)

In [14]:
# gray.shape

In [15]:
# plt.imshow(random_transform(gray),cmap='gray')
# plt.show()

# All images (if small) can be held in memory.

In [58]:
file_list = data['Image']

In [59]:
def get_image(file, shape=(resize_shape[0],resize_shape[1])):
    image = Image.open('data/train/' + file)
    image = image.resize(shape)
    image = np.array(image)
    if len(image.shape) == 2:
        image = np.stack([image]*3,axis=2) 
    return image

In [60]:
image_list = [get_image(f) for f in file_list]

In [61]:
length_list = [len(image.shape) for image in image_list]

In [62]:
file_list[0]

'00022e1a.jpg'

In [63]:
data['image_array'] = image_list

In [64]:
data.head(4)

,Image,Id,image_array
0,00022e1a.jpg,w_e15442c,"[[[196, 196, 196], [191, 191, 191], [195, 195,..."
1,000466c4.jpg,w_1287fbc,"[[[196, 189, 183], [194, 187, 181], [198, 191,..."
2,00087b01.jpg,w_da2efe0,"[[[191, 191, 191], [192, 192, 192], [186, 186,..."
3,001296d5.jpg,w_19e5482,"[[[119, 140, 219], [121, 142, 221], [122, 143,..."


# Create Test and Train

In [65]:
from sklearn.utils import shuffle
data = shuffle(data)

test_proportion = 0.8
cutoff_index = int(len(data) * test_proportion)

training_data = data.iloc[:cutoff_index].reset_index(drop=True)
test_data = data.iloc[cutoff_index:].reset_index(drop=True)

In [66]:
from collections import Counter

training_counts = Counter(training_data['Id'])
training_data['Id_count'] = training_data.apply(lambda x: training_counts.get(x["Id"]), axis=1)

test_counts = Counter(test_data['Id'])
test_data['Id_count'] = test_data.apply(lambda x: test_counts.get(x["Id"]), axis=1)

In [67]:
from random import randint

# Create generator

In [68]:
def get_triple(data):
#     filtered = data[data['Id_count'] > 1].reset_index(drop=True)
    filtered = data[(data.Id_count > 1) & (data.Id != 'new_whale')].reset_index(drop=True)
    
    anchor_index = randint(0,len(filtered)-1)
    anchor_image = filtered['image_array'][anchor_index]
    anchor_id = filtered['Id'][anchor_index]   
    same_id = anchor_id
    relevant_indices = list(filtered.index[filtered['Id'] == anchor_id])
    same_index = np.random.choice(relevant_indices)
    same_image = filtered['image_array'][same_index]
    
    different_id = anchor_id
    while (anchor_id == different_id):
        different_index = randint(0,len(data)-1)
        different_id = data['Id'][different_index]
    
    different_image = data['image_array'][different_index]
    anchor_image = random_transform(anchor_image)
    same_image = random_transform(same_image)
    different_image = random_transform(different_image)
    return anchor_image, same_image, different_image

In [69]:
def triple_generator(batch_size, data, resize_shape):
    while True:
        anchor_batch = np.zeros((batch_size, resize_shape[0],resize_shape[1],resize_shape[2]))
        same_image_batch = np.zeros((batch_size, resize_shape[0],resize_shape[1],resize_shape[2]))
        different_image_batch = np.zeros((batch_size, resize_shape[0],resize_shape[1],resize_shape[2]))
        for i in range(batch_size):
                anchor_batch[i,:,:,:], same_image_batch[i,:,:,:], different_image_batch[i,:,:,:] = get_triple(data)

        batches = [anchor_batch, same_image_batch, different_image_batch]
        yield batches, np.ones(batch_size)

# Create the network

In [28]:
# def L2_distance(X):

#     encoded_l, encoded_r = X

#     # BPR loss
#     loss = 1.0 - K.sigmoid(
#         K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
#         K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True))

#     return loss

In [29]:
input_shape = resize_shape

anchor_input = Input(input_shape)
same_category_input = Input(input_shape)
different_category_input = Input(input_shape)

In [74]:
def bpr_triplet_loss(X):    
    alpha = 3

    anchor_embedding, same_embedding, different_embedding = X
    
    positive_distance = K.square(anchor_embedding - same_embedding)
    negative_distance = K.square(anchor_embedding - different_embedding)
    
    positive_distance = K.mean(positive_distance, axis=-1, keepdims=True)
    negative_distance = K.mean(negative_distance, axis=-1, keepdims=True)
    
    loss = K.maximum(0.0, alpha + positive_distance - negative_distance)
   
    return K.mean(loss)

#Something like this:
# def triplet_loss(inputs, dist='sqeuclidean', margin='maxplus'):
#     anchor, positive, negative = inputs
#     positive_distance = K.square(anchor - positive)
#     negative_distance = K.square(anchor - negative)
#     if dist == 'euclidean':
#         positive_distance = K.sqrt(K.sum(positive_distance, axis=-1, keepdims=True))
#         negative_distance = K.sqrt(K.sum(negative_distance, axis=-1, keepdims=True))
#     elif dist == 'sqeuclidean':
#         positive_distance = K.mean(positive_distance, axis=-1, keepdims=True)
#         negative_distance = K.mean(negative_distance, axis=-1, keepdims=True)
#     loss = positive_distance - negative_distance
#     if margin == 'maxplus':
#         loss = K.maximum(0.0, 1 + loss)
#     elif margin == 'softplus':
#         loss = K.log(1 + K.exp(loss))
#     return K.mean(loss)



convnet = Sequential()
convnet.add(Conv2D(filters=8, input_shape=(
    resize_shape[0],resize_shape[1],resize_shape[2],), kernel_size=5, activation='relu',name='conv_1'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_1'))
convnet.add(Conv2D(filters=12, kernel_size=3, activation='relu',name='conv_2'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_2'))
convnet.add(Conv2D(filters=16, kernel_size=3, activation='relu',name='conv_3'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_3'))
convnet.add(Conv2D(filters=20, kernel_size=3, activation='relu',name='conv_4'))
convnet.add(Conv2D(filters=32, kernel_size=3, activation='relu',name='conv_5'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_4'))
convnet.add(Flatten())
convnet.add(Dense(units=128, activation='relu',name='dense_1'))
convnet.add(Dense(units=128, activation='relu',name='dense_2'))
convnet.add(Dense(units=64, activation='relu',name='dense_3'))

anchor = convnet(anchor_input)
same = convnet(same_category_input)
different = convnet(different_category_input)

loss = merge(
        [anchor, same, different],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(1, ))
    
siamese_net = Model(input=[anchor_input,same_category_input, different_category_input],output=loss)

/home/edward_bartrum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/edward_bartrum/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/edward_bartrum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("lo...)`


In [ ]:
def identity_loss(y_true, y_pred):
    return K.mean(y_pred - 0 * y_true)

In [100]:
LEARNING_RATE = 0.0000000000001
# siamese_net.compile(loss=identity_loss,optimizer=SGD(LEARNING_RATE))
siamese_net.compile(loss=identity_loss,optimizer=Adam())

# Run the network

In [127]:
from keras.optimizers import Adam, SGD

In [128]:
BATCH_SIZE = 512
training_data_generator = triple_generator(BATCH_SIZE, training_data, resize_shape)

In [ ]:
history = siamese_net.fit_generator(training_data_generator,
                                    verbose=1, 
                                    epochs=100, 
                                    steps_per_epoch=10,
                                    workers=16,
                                    use_multiprocessing=True)

/home/edward_bartrum/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:1987: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/100
10/10 [==============================] - 65s - loss: 0.4289    
Epoch 2/100
10/10 [==============================] - 44s - loss: 0.4041    
Epoch 3/100
10/10 [==============================] - 15s - loss: 0.4462    
Epoch 4/100
10/10 [==============================] - 43s - loss: 0.4328    
Epoch 5/100
10/10 [==============================] - 44s - loss: 0.4228    
Epoch 6/100
 3/10 [========>.....................] - ETA: 11s - loss: 0.4934

# Loss function 

In [ ]:
plt.plot(history.history['loss'])
plt.title("Loss function")
plt.xlabel("epochs")
plt.show()

# Evaluation 

In [ ]:
evaluation_data_generator = triple_generator(BATCH_SIZE, test_data, resize_shape)
evaluation_steps = 20
metric_names = siamese_net.metrics_names
metric_values = siamese_net.evaluate_generator(evaluation_data_generator, steps=evaluation_steps)
print("Metric names", metric_names)
print("Metric values", metric_values)

# Save weights

In [ ]:
import time
import os

now = time.strftime('%Y.%m.%d %H:%M:%S')
directory = "weights/" + now + "/"
if not os.path.exists(directory):
    os.makedirs(directory)


siamese_net.save_weights(directory + "siamese_weights")
convnet.save_weights(directory + "convnet_weights")

# Sub

In [ ]:
data = pd.read_csv("data/train.csv")
file_list = data['Image']
image_list = [get_image(f) for f in file_list]

In [ ]:
id_list = data['Id']

In [ ]:
embedding_list = convnet.predict(np.stack(image_list))

In [ ]:
def get_sub_image(file, shape=(resize_shape[0],resize_shape[1])):
    image = Image.open('data/test/' + file)
    image = image.resize(shape)
    image = np.array(image)
    if len(image.shape) == 2:
        image = np.stack([image]*3,axis=2) 
    return image


sample_sub = pd.read_csv("data/sample_submission.csv")
submission_file_list = sample_sub['Image']
submission_image_list = [get_sub_image(f) for f in submission_file_list]

In [ ]:
submission_embedding_list = convnet.predict(np.stack(submission_image_list))

In [ ]:
X = [1,2,3,4]
Y = [9,8,2,3]

[x for (y,x) in sorted(zip(Y,X), key=lambda pair: pair[0])]

In [ ]:
def remove_duplicates(li):
    my_set = set()
    filtered = []
    for e in li:
        if e not in my_set:
            filtered.append(e)
            my_set.add(e)
    return filtered


def classify(image_embedding, embedding_list, id_list, num_categories=5):
    image_embedding = np.expand_dims(image_embedding, axis=0)
    stacked_image = np.repeat(image_embedding,len(embedding_list),axis=0)
    square_differences = (stacked_image - embedding_list)**2
    scores = np.sum(square_differences, axis=1)    
    sorted_ids = [x for (y,x) in sorted(
        zip(scores,id_list), key=lambda pair: pair[0])]
    
    return ' '.join(remove_duplicates(sorted_ids[0:num_categories]))
    

In [ ]:
classify(embedding_list[7], embedding_list, id_list)

In [ ]:
data.head(10)

In [ ]:
submission_embedding_list

In [ ]:
submission_prediction_list = [classify(image_embedding, embedding_list, id_list) for image_embedding in submission_embedding_list]

In [ ]:
submission = pd.DataFrame({'Image': submission_file_list, 'Id': submission_prediction_list}, columns=['Image','Id'])

In [ ]:
submission.head(10)

In [ ]:
submission.to_csv("results/submission.csv",index=False)

In [ ]:
np.sort(id_list)